# Interactive Explorer for Harmonic Field Consciousness Model

This notebook provides interactive widgets to explore the consciousness model parameters and visualize results in real-time.

In [ ]:
# Imports
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, FloatSlider, IntSlider, Dropdown
import ipywidgets as widgets

from utils import graph_generators as gg
from utils import metrics as met
from utils import state_generators as sg
from utils import visualization as viz

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Explore Network Topologies

In [ ]:
def explore_networks(topology='small_world', n_nodes=100, k_neighbors=6, rewiring_prob=0.3):
    """Interactive network explorer."""
    
    # Generate network based on topology
    if topology == 'small_world':
        G = gg.generate_small_world(n_nodes, k_neighbors, rewiring_prob, seed=42)
    elif topology == 'scale_free':
        G = gg.generate_scale_free(n_nodes, m_edges=3, seed=42)
    elif topology == 'random':
        G = gg.generate_random(n_nodes, edge_prob=0.06, seed=42)
    elif topology == 'lattice':
        G = gg.generate_lattice(n_nodes, dimension=2, seed=42)
    else:
        G = gg.generate_modular(n_nodes, n_modules=4, seed=42)[0]
    
    # Compute eigenmodes
    L, eigenvalues, eigenvectors = gg.compute_laplacian_eigenmodes(G)
    
    # Visualize
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Network structure
    ax = axes[0]
    import networkx as nx
    pos = nx.spring_layout(G, seed=42)
    nx.draw_networkx_edges(G, pos, ax=ax, alpha=0.3, edge_color='gray')
    nx.draw_networkx_nodes(G, pos, ax=ax, node_size=50, node_color='steelblue')
    ax.set_title(f'{topology.upper()}: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges')
    ax.axis('off')
    
    # Eigenvalue spectrum
    ax = axes[1]
    ax.plot(eigenvalues[:30], 'o-', markersize=6)
    ax.set_xlabel('Mode index')
    ax.set_ylabel('Eigenvalue')
    ax.set_title('Eigenvalue Spectrum (first 30 modes)')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Create interactive widget
interact(explore_networks,
         topology=Dropdown(options=['small_world', 'scale_free', 'random', 'lattice', 'modular'],
                          value='small_world', description='Topology:'),
         n_nodes=IntSlider(min=50, max=200, step=10, value=100, description='Nodes:'),
         k_neighbors=IntSlider(min=2, max=10, step=1, value=6, description='K neighbors:'),
         rewiring_prob=FloatSlider(min=0, max=1, step=0.1, value=0.3, description='Rewiring p:'));

## 2. Explore Brain States

In [ ]:
def explore_states(state='wake', n_modes=30, anesthesia_depth=1.0, psychedelic_intensity=0.5):
    """Interactive brain state explorer."""
    
    # Generate state
    if state == 'wake':
        power = sg.generate_wake_state(n_modes, seed=42)
    elif state == 'nrem':
        power = sg.generate_nrem_unconscious(n_modes, seed=42)
    elif state == 'dream':
        power = sg.generate_nrem_dreaming(n_modes, seed=42)
    elif state == 'anesthesia':
        power = sg.generate_anesthesia_state(n_modes, depth=anesthesia_depth, seed=42)
    else:
        power = sg.generate_psychedelic_state(n_modes, intensity=psychedelic_intensity, seed=42)
    
    # Compute metrics
    eigenvalues = np.linspace(0, 10, n_modes)
    metrics = met.compute_all_metrics(power, eigenvalues)
    
    # Visualize
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Power distribution
    ax = axes[0]
    k = np.arange(n_modes)
    ax.bar(k, power, color='steelblue', alpha=0.7, edgecolor='black')
    ax.set_xlabel('Mode index')
    ax.set_ylabel('Power')
    ax.set_title(f'{state.upper()} State: Mode Power Distribution')
    ax.grid(True, alpha=0.3, axis='y')
    
    # Metrics radar chart
    ax = axes[1]
    categories = ['H_mode', 'PR', 'R', 'S_dot', 'kappa']
    values = [metrics[cat] for cat in categories]
    values += values[:1]  # Close the circle
    
    angles = np.linspace(0, 2 * np.pi, len(categories), endpoint=False).tolist()
    angles += angles[:1]
    
    ax = plt.subplot(122, projection='polar')
    ax.plot(angles, values, 'o-', linewidth=2)
    ax.fill(angles, values, alpha=0.25)
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(categories)
    ax.set_ylim(0, 1)
    ax.set_title(f'Consciousness Metrics\nC(t) = {metrics["C"]:.3f}', y=1.08)
    ax.grid(True)
    
    plt.tight_layout()
    plt.show()

# Create interactive widget
interact(explore_states,
         state=Dropdown(options=['wake', 'nrem', 'dream', 'anesthesia', 'psychedelic'],
                       value='wake', description='State:'),
         n_modes=IntSlider(min=10, max=50, step=5, value=30, description='N modes:'),
         anesthesia_depth=FloatSlider(min=0, max=1, step=0.1, value=1.0, description='Anest. depth:'),
         psychedelic_intensity=FloatSlider(min=0, max=1, step=0.1, value=0.5, description='Psych. int.:'));

## 3. Compare States

In [ ]:
def compare_states():
    """Compare all brain states."""
    n_modes = 30
    eigenvalues = np.linspace(0, 10, n_modes)
    
    states = {
        'Wake': sg.generate_wake_state(n_modes, seed=42),
        'NREM': sg.generate_nrem_unconscious(n_modes, seed=42),
        'Dream': sg.generate_nrem_dreaming(n_modes, seed=42),
        'Anesthesia': sg.generate_anesthesia_state(n_modes, seed=42),
        'Psychedelic': sg.generate_psychedelic_state(n_modes, seed=42),
    }
    
    # Compute metrics
    metrics_all = {}
    for name, power in states.items():
        metrics_all[name] = met.compute_all_metrics(power, eigenvalues)
    
    # Plot comparison
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    
    # Power distributions
    for idx, (name, power) in enumerate(states.items()):
        ax = axes[idx // 3, idx % 3]
        k = np.arange(n_modes)
        ax.bar(k, power, alpha=0.7, edgecolor='black')
        ax.set_xlabel('Mode')
        ax.set_ylabel('Power')
        ax.set_title(f'{name}\nC(t)={metrics_all[name]["C"]:.3f}')
    
    # Comparison bar chart
    ax = axes[1, 2]
    names = list(states.keys())
    c_values = [metrics_all[name]['C'] for name in names]
    ax.bar(range(len(names)), c_values, alpha=0.7, edgecolor='black')
    ax.set_xticks(range(len(names)))
    ax.set_xticklabels(names, rotation=45, ha='right')
    ax.set_ylabel('C(t)')
    ax.set_title('Consciousness Functional')
    ax.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()

compare_states()

## 4. Export Configuration

Export interesting parameter configurations for use in experiments.

In [ ]:
import json

# Example configuration
config = {
    'network': {
        'topology': 'small_world',
        'n_nodes': 100,
        'k_neighbors': 6,
        'rewiring_prob': 0.3,
    },
    'states': {
        'n_modes': 30,
    },
    'seed': 42,
}

# Save
with open('config_export.json', 'w') as f:
    json.dump(config, f, indent=2)

print("Configuration exported to config_export.json")